In [ ]:
!pip install trl peft bitsandbytes

In [2]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
import pandas as pd
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from datasets import Dataset
import random

from trl import DPOConfig

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [24]:
df_ar_ft = pd.read_csv("ar-dataset-ft.csv")

df_ar = pd.read_csv("ar-dataset.csv")

2648

In [26]:
chosen_df = df_ar[df_ar['can_be_guessed'] == True]
rejected_df = df_ar[df_ar['can_be_guessed'] == False]

# Balance the size of chosen and rejected DataFrames
# if len(rejected_df) > len(chosen_df):
#     rejected_df = rejected_df.sample(len(chosen_df), random_state=42)
# elif len(chosen_df) > len(rejected_df):
#     chosen_df = chosen_df.sample(len(rejected_df), random_state=42)

user_prompts = [
    "Please create a medical multiple-choice question with four possible answers, only one correct.",
    "Generate a single medical multiple-choice question with exactly one correct answer.",
    "I need a medical multiple-choice question (4 options) with one correct answer.",
    "Write a medical MCQ with four answer choices, only one of which is correct.",
    "Produce a medical multiple-choice question with four options and identify a single correct choice.",
]

In [31]:
# Sample evaluation data
eval_chosen_indices = chosen_df.sample(n=150, random_state=42).index
eval_rejected_indices = rejected_df.sample(n=150, random_state=42).index

eval_chosen_df = chosen_df.loc[eval_chosen_indices].reset_index(drop=True)
eval_rejected_df = rejected_df.loc[eval_rejected_indices].reset_index(drop=True)

# Remaining rows for training
train_chosen_df = chosen_df.drop(eval_chosen_indices).reset_index(drop=True)
train_rejected_df = rejected_df.drop(eval_rejected_indices).reset_index(drop=True)

# Function to create pairs
def create_pair(chosen_row, rejected_row):
    chosen_text = (
        f"Question: {chosen_row['question']}\n"
        f"a) {chosen_row['option_a']}\n"
        f"b) {chosen_row['option_b']}\n"
        f"c) {chosen_row['option_c']}\n"
        f"d) {chosen_row['option_d']}"
    )
    rejected_text = (
        f"Question: {rejected_row['question']}\n"
        f"a) {rejected_row['option_a']}\n"
        f"b) {rejected_row['option_b']}\n"
        f"c) {rejected_row['option_c']}\n"
        f"d) {rejected_row['option_d']}"
    )
    return {
        "chosen": chosen_text,
        "rejected": rejected_text
    }

# Create eval_df with unique pairs
eval_examples = []
for i in range(len(eval_chosen_df)):
    chosen_row = eval_chosen_df.iloc[i]
    rejected_row = eval_rejected_df.iloc[i]
    eval_examples.append(create_pair(chosen_row, rejected_row))

eval_df = pd.DataFrame(eval_examples)

# Upsample train_chosen_df to match train_rejected_df size
upsampled_train_chosen_df = train_chosen_df.sample(n=len(train_rejected_df), replace=True, random_state=42).reset_index(drop=True)

# Create training dataset allowing duplicates in chosen_df but ensuring no intersection with eval_df
training_examples = []

train_rejected_df = train_rejected_df.sample(frac=1, random_state=42).reset_index(drop=True)
upsampled_train_chosen_df = upsampled_train_chosen_df.sample(frac=1, random_state=42).reset_index(drop=True)

for i in range(len(upsampled_train_chosen_df)):
    chosen_row = upsampled_train_chosen_df.iloc[i]
    rejected_row = train_rejected_df.iloc[i % len(train_rejected_df)]
    training_examples.append(create_pair(chosen_row, rejected_row))

train_df = pd.DataFrame(training_examples)

# Convert to Hugging Face Dataset format if needed
eval_dataset = Dataset.from_pandas(eval_df)
train_dataset = Dataset.from_pandas(train_df)

In [34]:
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
# )
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules=['o_proj', 'qkv_proj'] #phi-3
    target_modules="all-linear"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    quantization_config=bnb_config
)
model.config.use_cache = False



# # Model to fine-tune
# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3.5-mini-instruct",
#     torch_dtype=torch.float16,
#     load_in_4bit=True
# )
# model.config.use_cache = False

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [39]:
# training_args = DPOConfig(
#     num_train_epochs=5,
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     gradient_checkpointing=True,
#     learning_rate=5e-5,
#     lr_scheduler_type="cosine",
#     # max_steps=40,
#     save_strategy="no",
#     logging_steps=10,
#     output_dir="new_model",
#     optim="paged_adamw_32bit",
#     warmup_steps=10,
#     fp16=True,
#     do_train=True,
#     report_to=["wandb"]
# )
new_model = "model"

training_args = DPOConfig(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    remove_unused_columns=True,
    learning_rate=5.0e-06,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    num_train_epochs=8,
    save_strategy="epoch",
    logging_steps=10,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    bf16=True,
    report_to="none",
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
# dpo_trainer = DPOTrainer(
#     model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=tokenizer,
#     peft_config=peft_config,
#     beta=0.1,
#     max_prompt_length=1024,
#     max_length=1536
# )

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=2048,
    max_length=2048,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:475: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(


Extracting prompt from train dataset:   0%|          | 0/2498 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2498 [00:00<?, ? examples/s]

Extracting prompt from eval dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2498 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

In [41]:
dpo_trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,0.611300,0.665611,-0.147208,-0.237709,0.640351,0.090501,-180.897995,-162.675797,31.470928,31.312637
1,0.491800,0.722329,-0.972448,-1.060719,0.625000,0.088270,-189.150421,-170.905884,30.377962,30.155954
2,0.319100,0.805584,-1.554435,-1.637492,0.589912,0.083057,-194.970261,-176.673599,29.433735,29.138147
4,0.229800,0.963161,-2.496490,-2.539704,0.543860,0.043213,-204.390823,-185.695724,27.585995,27.172632
5,0.217100,1.042253,-2.914506,-2.925185,0.550439,0.010680,-208.570984,-189.550522,26.848993,26.386171
6,0.169400,1.048652,-2.978190,-2.992708,0.543860,0.014518,-209.207840,-190.225769,26.766136,26.280542
7,0.117900,1.041938,-2.973210,-2.997333,0.550439,0.024123,-209.158020,-190.272018,26.775774,26.294468


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn

TrainOutput(global_step=1248, training_loss=0.32188514753794056, metrics={'train_runtime': 3543.959, 'train_samples_per_second': 5.639, 'train_steps_per_second': 0.352, 'total_flos': 0.0, 'train_loss': 0.32188514753794056, 'epoch': 7.9872})

In [ ]:
!zip -r checkpoint_ar_390.zip /content/model/checkpoint-390